# Explore alternative circuits
In this tutorial, we explore alternative circuits that can generate the target quantum state by utilizing the set of isomorphic graphs.

In [8]:
from graphiq.backends.lc_equivalence_check import iso_graph_finder
from graphiq.backends.stabilizer.compiler import StabilizerCompiler
from graphiq.state import QuantumState
from graphiq.metrics import Infidelity, CircuitDepth
from graphiq.solvers.time_reversed_solver import TimeReversedSolver

import networkx as nx

Again, we target a 4-qubit linear cluster state and use utility functions for generating the set of isomorphic graphs to our target -- defined as a `networkx` graph.

In [13]:
compiler = StabilizerCompiler()
graph = nx.Graph([(0, 1), (1, 2), (2, 3)])
iso_graphs = iso_graph_finder(graph)
print(f"{len(iso_graphs)} graphs that are isomorphic to the input.")

24 graphs that are isomorphic to the input.


We first check the circuit depth of the generating circuit identified using the `TimeReversedSolver` for the original target. 

In [21]:
target = QuantumState(graph, rep_type="graph")
metric = CircuitDepth()
solver = TimeReversedSolver(
    target=target,
    metric=metric,
    compiler=compiler,
)
solver.solve()
best_score, circuit = solver.result

print(f"Starting graph has depth {best_score}.")

Starting graph has depth 9.


We then loop over all graphs that are isomorphic to our original target, compute a generating circuit, and check the circuit depth for all. In this way, we exhaustively look for a generating circuit that has lower depth.

In [22]:
for graph in iso_graphs:
    solver = TimeReversedSolver(
        target=QuantumState(graph, rep_type="graph"),
        metric=metric,
        compiler=compiler,
    )
    solver.solve()
    score, circuit = solver.result
    if score < best_score:
        best_score = score

print(f"Best score is {best_score}")

Best score is 6
